```
%%bash
source /data/reddylab/software/miniconda3/bin/activate alex_py3

sbatch -pall \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem=48G \
    --output /data/reddylab/Alex/encode4_duke/logs/wgstarr.input.bam_to_bedpe_to_frag_count.dedup.%a.out \
    --array=0-3 \
    <<'EOF'
#!/bin/bash
BAMS=(\
    /data/reddylab/kstrouse/superstarr/input_libs/A001/nextseq/processing/starr_seq/A001_nextseq-pe/rep1.f3q10.sorted.dups_marked.bam \
    /data/reddylab/kstrouse/superstarr/input_libs/A001/nextseq/processing/starr_seq/A001_nextseq-pe/rep2.f3q10.sorted.dups_marked.bam \
    /data/reddylab/kstrouse/superstarr/input_libs/A001/nextseq/processing/starr_seq/A001_nextseq-pe/rep3.f3q10.sorted.dups_marked.bam \
    /data/reddylab/kstrouse/superstarr/input_libs/A001/nextseq/processing/starr_seq/A001_nextseq-pe/rep4.f3q10.sorted.dups_marked.bam \
)
DATA_DIR="/data/reddylab/Alex/encode4_duke/data/starr_seq/fragments"
mkdir -p ${DATA_DIR}
# SAMPLES=($(cut -f3 /data/reddylab/Alex/encode4_duke//data/starr_seq/metadata/starr_seq_download_metadata.210401_KS91_K562ASTARR_NovaSeq.txt | tail -n+2 | cut -d_ -f1-6 | sort -u ))
BAM=${BAMS[${SLURM_ARRAY_TASK_ID}]}
SAMPLE="A001-input-K562-rep$((${SLURM_ARRAY_TASK_ID}+1))"
OUTDIR=${DATA_DIR}

#     --region chr11:33906524:33964898 \
#     --ignoreDuplicates \
samtools view \
    -b \
    -q 10 \
    -F 1024 \
    ${BAM} \
| samtools sort -n \
    -@ 8 \
    -T ${OUTDIR}/${SAMPLE}.tmp \
| bedtools bamtobed \
    -bedpe \
    -i - \
| awk -vOFS="@" '{if($2>$5){TMP=$2; $2=$5; $5=TMP}; if($3>$6){TMP=$3; $3=$6; $6=TMP}print $1, $2, $6}' \
| sort --parallel 8 \
| uniq -c \
| awk -vOFS="\t" '{print $2, $1}' \
| sed 's/@/\t/g' \
| sort -k1,1 -k2,2n \
| awk -vOFS="\t" '{print $1, $2, $3, $1"_"$2"_"$3, $4, "."}' \
| gzip -c \
> ${OUTDIR}/${SAMPLE}.masked.dedup.fragments.counts.txt.gz \
&& echo -e "Done!\t${SAMPLE}.masked.dedup.fragments.counts.txt.gz" \
|| echo -e "Failed!\t${SAMPLE}.masked.dedup.fragments.counts.txt.gz"

# | gzip -c \
# > ${OUTDIR}/${SAMPLE}.masked.dedup.fragments.bedpe.gz \
# && echo -e "Done!\t${OUTDIR}/${SAMPLE}.masked.dedup.fragments.bedpe.gz" \
# || echo -e "Failed!\t${OUTDIR}/${SAMPLE}.masked.dedup.fragments.bedpe.gz"

# gzip -dc ${OUTDIR}/${SAMPLE}.masked.dedup.fragments.bedpe.gz \


EOF
```